In [ ]:
!pip install datasets
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is i

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')  # Mounts Google Drive

Mounted at /content/gdrive


In [ ]:
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
from datasets import load_dataset
import re
import numpy as np
from tqdm import tqdm
import torch
import random
import pickle
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from collections import defaultdict

In [ ]:
def get_dataset():
    train_dataset = load_dataset("openai/gsm8k", "main", split='train')
    test_dataset = load_dataset("openai/gsm8k", "main", split='test')
    return train_dataset, test_dataset

In [ ]:
def get_model(model_name):
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True
    )
    if model_name == "wizardmath":
        wizardmath_tokenizer = AutoTokenizer.from_pretrained("WizardLM/WizardMath-7B-V1.1")
        wizardmath_model = AutoModelForCausalLM.from_pretrained(
            "WizardLM/WizardMath-7B-V1.1",
            quantization_config=quantization_config,
            device_map={"": 0},
            torch_dtype=torch.float16
        )
        return {
            'model': wizardmath_model,
            'model_name': "wizardmath",
            'tokenizer': wizardmath_tokenizer,
            'cost_per_token': 0.7
        }
    elif model_name == "phi2":
        phi2_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)
        phi2_model = AutoModelForCausalLM.from_pretrained(
            "microsoft/phi-2",
            quantization_config=quantization_config,
            device_map={"": 0},
            torch_dtype=torch.float16,
            trust_remote_code=True
        )
        return {
            'model': phi2_model,
            'model_name': "phi2",
            'tokenizer': phi2_tokenizer,
            'cost_per_token': 0.13
        }

In [ ]:
def extract_answer(answer_text):
    # The final answer in GSM8K follows the '####' pattern
    match = re.search(r'####\s*(-?\d+)', answer_text)
    if match:
        return match.group(1).strip()
    return None

In [ ]:
def process_problem(problem, model_index, models):
    prompt = f"""

Follow these instructions:
1. Work through the problem step by step
2. Calculate the numerical answer
3. On the last line, write ONLY: #### <numerical answer>. Do not add any units like "kg" or "m", or any currency symbols like "$".
4. Do not write anything after the final answer

-------------------
EXAMPLE FORMAT:
Step 1: [explanation]
Step 2: [explanation]
Final calculation: [calculation]
#### [numerical answer]
-------------------

NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}
"""
    # print("Entered global process problem")
    model_obj = models[model_index]['model']
    tokenizer = models[model_index].get('tokenizer', None)
    if tokenizer:
        tokenizer = models[model_index]['tokenizer']

    # if models[model_index]['model_name'] == "wizardmath":
    inputs = tokenizer(prompt, return_tensors="pt").to(model_obj.device)
    outputs = model_obj.generate(
        inputs.input_ids,
        max_new_tokens=1024,
        temperature=0.1,
        do_sample=True,
        attention_mask=inputs.attention_mask,
        # pad_token_id=tokenizer.eos_token_id,
    )
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    prompt_end = full_output.find(f"NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}")
    if prompt_end != -1:
        prompt_end = prompt_end + len(f"NOW SOLVE THE PROBLEM CORRECTLY: {problem['question']}")
        model_response = full_output[prompt_end:].strip()
    else:
        model_response = full_output

    hash_match = re.search(r'####\s*([\$]?\s*\d+(?:\.\d+)?)', model_response)
    if hash_match:
        answer_text = hash_match.group(1)
        numeric_match = re.search(r'(\d+(?:\.\d+)?)', answer_text)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            # return f"{prompt}\n\n{model_response.split('####')[0].strip()}\n#### {numeric_answer}"
            return {
                'prompt': prompt,
                'response': model_response,
                'answer': numeric_answer
            }

    answer_match = re.search(r'(?:final answer|the answer is)[^0-9]*?([\$]?\s*\d+(?:\.\d+)?)',
                            model_response.lower())
    if answer_match:
        answer_text = answer_match.group(1)
        numeric_match = re.search(r'(\d+(?:\.\d+)?)', answer_text)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            answer_position = model_response.lower().find(answer_match.group(0))
            if answer_position != -1:
                # return f"{prompt}\n\n{model_response[:answer_position].strip()}\n#### {numeric_answer}"
                return {
                    'prompt': prompt,
                    'response': model_response,
                    'answer': numeric_answer
                }

    lines = model_response.split('\n')
    for i in range(len(lines)-1, max(0, len(lines)-5), -1):
        line = lines[i]
        if len(line.strip()) < 1 or any(word in line.lower() for word in ["step", "explanation"]):
            continue

        numeric_match = re.search(r'(\d+(?:\.\d+)?)', line)
        if numeric_match:
            numeric_answer = numeric_match.group(1)
            # return f"{prompt}\n\n{model_response.split(line)[0].strip()}\n#### {numeric_answer}"
            return {
                'prompt': prompt,
                'response': model_response,
                'answer': numeric_answer
            }

    # return full_output
    return {
        'prompt': prompt,
        'response': full_output,
        'answer': None
    }


In [ ]:
temp_set = get_dataset()
gsm8k_dataset = {
    'train': temp_set[0],
    'test': temp_set[1]
}
models = [get_model('phi2'), get_model('wizardmath')]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
import pandas as pd
import time

diff_df = pd.read_csv("gsm8k_difficulty_preds.csv")

In [ ]:
total_correct = 0
total_cost = 0

num_problems = 500
cur_problem_idx = 0
start_idx = 2001
subset = gsm8k_dataset['train'].select(range(start_idx, start_idx+num_problems))
phi2_preds = []
wizardmath_preds = []

start_time = time.time()
for problem in tqdm(subset):
    # model_idx = 1 if diff_df.iloc[cur_problem_idx + start_idx]['label'] == False else 0
    target_index = start_idx + cur_problem_idx
    label_value = diff_df.loc[diff_df['index'] == target_index, 'label'].item()
    model_idx = 1 if not label_value else 0
    # print(model_idx)
    cur_problem_idx += 1

    # print(problem)
    correct_answer = extract_answer(problem['answer'])
    prediction = process_problem(problem, model_idx, models)
    predicted_answer = prediction['answer']
    # print(f"Correct Answer: {correct_answer}")
    # print(f"Predicted Answer: {predicted_answer}")
    # print(prediction['response'])
    if predicted_answer is not None and float(predicted_answer) == float(correct_answer):
        total_correct += 1
        if model_idx == 0:
            phi2_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': True})
        else:
            wizardmath_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': True})
    else:
        if model_idx == 0:
            phi2_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': False})
        else:
            wizardmath_preds.append({'question': problem['question'], 'answer': problem['answer'], 'model_response': prediction['response'], 'is_correct': False})

end_time = time.time()

100%|██████████| 500/500 [4:01:31<00:00, 28.98s/it]


In [ ]:
# check if /content/gdrive/MyDrive/ai_project exists, if not create it
import os
if not os.path.exists('/content/gdrive/MyDrive/ai_project'):
    os.makedirs('/content/gdrive/MyDrive/ai_project')

In [ ]:
import pandas as pd
import os

os.makedirs('/content', exist_ok=True)

phi2_df = pd.DataFrame(phi2_preds)
phi2_df['model_name'] = 'phi-2'

wizardmath_df = pd.DataFrame(wizardmath_preds)
wizardmath_df['model_name'] = 'wizardmath'

# Combine and save
combined_df = pd.concat([phi2_df, wizardmath_df], ignore_index=True)
# combined_csv_path = '/content/combined_predictions.csv'
# combined_df.to_csv(combined_csv_path, index=False)
# Save files to Drive instead of /content
combined_csv_path = '/content/gdrive/MyDrive/ai_project/combined_predictions.csv'
combined_df.to_csv(combined_csv_path, index=False)

accuracy = total_correct / num_problems
time_taken = end_time - start_time

# Create metrics DataFrame
metrics_df = pd.DataFrame({
    'total_problems': [num_problems],
    'correct_answers': [total_correct],
    'accuracy': [accuracy],
    'time_seconds': [time_taken],
    'avg_time_per_problem': [time_taken/num_problems],
    'total_cost': [total_cost]
})

# metrics_csv_path = '/content/experiment_metrics.csv'
metrics_csv_path = '/content/gdrive/MyDrive/ai_project/experiment_metrics.csv'
metrics_df.to_csv(metrics_csv_path, index=False)

fpath = '/content/gdrive/MyDrive/ai_project/accuracy_summary.txt'
# with open('/content/accuracy_summary.txt', 'w') as f:
with open(fpath, 'w') as f:
    f.write(f"""Experiment Results:
Total Problems: {num_problems}
Correct Answers: {total_correct}
Accuracy: {accuracy:.2%}
Time Taken: {time_taken:.2f} seconds
Average Time per Problem: {time_taken/num_problems:.2f}s
Total Cost: ${total_cost:.4f}
""")

print(f"Predictions saved to: {combined_csv_path}")
print(f"Metrics saved to: {metrics_csv_path}")


Predictions saved to: /content/gdrive/MyDrive/ai_project/combined_predictions.csv
Metrics saved to: /content/gdrive/MyDrive/ai_project/experiment_metrics.csv


In [ ]:
(total_correct/num_problems)*100

74.2